# Monkeypox KG


# Package installation is required only once

In [ ]:
pip install -r packages.txt

In [1]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
import chembl_webresource_client
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from ByCovidGraph import *
import pubchempy as pcp

C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\pybel\struct\operations.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# drugs against monkeypox from https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12

mpox = 'CHEMBL152 CHEMBL203321 CHEMBL1257073'.split()
mpox

['CHEMBL152', 'CHEMBL203321', 'CHEMBL1257073']

In [7]:
#reading file from pubchem for monkeypox 
#https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244

mpox_pchem = pd.read_csv('data/normalized_data/TaxID_10244_bioactivity.csv')
mpox_pchem_cid = mpox_pchem['cid']
mpox_pchem_cid = list(set(mpox_pchem_cid))
mpox_pchem_cid

[134406,
 44420748,
 101879053,
 101879054,
 16124688,
 11243799,
 155545375,
 11773602,
 73124,
 37542,
 44341289,
 44341811,
 44341812,
 118895042,
 118895044,
 60613,
 11301322,
 189926,
 5271015,
 11452011,
 11463022,
 155521397,
 45111031,
 122873]

In [4]:
#Function to retrieve assay ids for compounds
def RetAct(chemblIds,j):
    GetAct = new_client.activity
    ActList = []
    #for i in range(len(chemblIds)):
    #print(chemblIds[0])
    for i in range(len(chemblIds)):
        filtered_list=['assay_chembl_id','assay_type','pchembl_value','target_chembl_id','target_organism','bao_label']
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,assay_type_iregex='(B|F)',target_organism='Homo sapiens').only(filtered_list)
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,target_organism='Homo sapiens').only(filtered_list)
        acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False)
        j=j+1

        #acts = [d for d in acts if d.get('target_organism') == 'Homo sapiens']
        #acts = [d for d in acts if d.get('bao_label') == 'single protein format']
        #acts = [d for d in acts if d.get('type') in ['Ki', 'IC50']]
        acts = [d for d in acts if float(d.get('pchembl_value')) >= 6]
        print(j)
        #print(len(acts))
        acts = acts[:5]
        print(acts)
        ActList.append(list(acts))
    #print(ActList)
    named_ActList = dict(zip(chemblIds,ActList))
    named_ActList = {k: v for k, v in named_ActList.items() if v}
    return(named_ActList)


In [8]:
#convert pubchem cids to chembl ids

cid2chembl_dict = []

for id in mpox_pchem_cid:
    #if id != 'None':
    print(id)
    GetChembl = {}
    c=pcp.Compound.from_cid(id)
    #print(id)
    syn = c.synonyms
    for s in syn:
        #if s.startswith('SCHEMBL'):
            #GetChembl['SCHEMBL'] = s
        if s.startswith('CHEMBL'):
            GetChembl['CHEMBL'] = s

    cid2chembl_dict.append(GetChembl)
        

134406
44420748
101879053
101879054
16124688
11243799
155545375
11773602
73124
37542
44341289
44341811
44341812
118895042
118895044
60613
11301322
189926
5271015
11452011
11463022
155521397
45111031
122873


In [17]:
chemblid = []

for id in cid2chembl_dict:
    #chemblid.append()
    #print(id.values())
    chembl = list(id.values())
    #print(chembl[0])
    chemblid.append(chembl[0])
    

In [18]:
chemblid = mpox+chemblid
len(chemblid)

27

In [19]:
pchem_mech = RetMech(chemblid)


[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[{'mechanism_of_action': 'DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL3988586'}]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[]
[]
[]
[]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[]
[]
[]
[]
[{'mechanism_of_action': "Inosine-5'-monophosphate dehydrogenase 1 inhibitor", 'target_chembl_id': 'CHEMBL1822'}, {'mechanism_of_action': 'Hepatitis C virus NS5B RNA-dependent RNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL5375'}, {'mechanism_of_action': 'RNA inhibitor', 'target_chembl_id': 'CHEMBL2363073'}]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[]
[]
[]
[]
[]
[]
[]
[]


In [20]:
pchem_dis = RetDrugInd(chemblid)

[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Cytomegalovirus Infections'}, {'mesh_heading': 'Hemorrhagic Fever, Ebola'}, {'mesh_heading': 'Adenovirus Infections, Human'}, {'mesh_heading': 'Adenoviridae Infections'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Smallpox'}]
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Smallpox'}]
[]
[]
[]
[]
[{'mesh_heading': 'Leukemia, Myeloid, Acute'}, {'mesh_heading': 'Hepatitis C, Chronic'}, {'mesh_heading': 'Hepatitis C'}, {'mesh_heading': 'Hepatitis B, Chronic'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[]
[]
[]
[]


In [21]:
pchem_act = RetAct(chemblid,0)

1
[{'activity_comment': None, 'activity_id': 302681, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL746888', 'assay_description': 'Inhibitory activity against replication of HCMV in NHDF cells', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000219', 'bao_label': 'cell-based format', 'canonical_smiles': 'Nc1ccn(C[C@@H](CO)OCP(=O)(O)O)c(=O)n1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1129529', 'document_journal': 'J. Med. Chem.', 'document_year': 1996, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL152', 'molecule_pref_name': 'CIDOFOVIR', 'parent_molecule_chembl_id': 'CHEMBL152', 'pchembl_value': '6.30', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 37715, 'relation': '=', 'src_id': 1, 'standard_flag': 1, 'standard_relation': '=', 'standard_text_value': None, 'standard_type': 'IC5

In [22]:
chemid2 = Ret_chembl_protein(pchem_act) + Ret_chembl_protein(pchem_mech)
chemid2

['CHEMBL613168',
 'CHEMBL612558',
 'CHEMBL2664',
 'CHEMBL612672',
 'CHEMBL614200',
 'CHEMBL371',
 'CHEMBL613493',
 'CHEMBL4434621',
 'CHEMBL613888',
 'CHEMBL340',
 'CHEMBL612545',
 'CHEMBL3751646',
 'CHEMBL613120',
 'CHEMBL613332',
 'CHEMBL3879801',
 'CHEMBL614149',
 'CHEMBL276',
 'CHEMBL613494',
 'CHEMBL1293235',
 'CHEMBL613738',
 'CHEMBL613495',
 'CHEMBL613167',
 'CHEMBL5375',
 'CHEMBL1872',
 'CHEMBL3988586',
 'CHEMBL2363073',
 'CHEMBL4296170',
 'CHEMBL1822']

In [23]:
chem2uni = chembl2uniprot(chemid2,0)
chem2uni

old 10
newLen 8
2
[{'xref_id': 'R-HSA-156581', 'xref_name': 'Methylation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-1614635', 'xref_name': 'Sulfur amino acid metabolism.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2408508', 'xref_name': 'Metabolism of ingested SeMet, Sec, MeSec into H2Se.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5578997', 'xref_name': 'Defective AHCY causes HMAHCHD.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'AHCY', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-211981', 'xref_name': 'Xenobiotics.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5423646', 'xref_name': 'Aflatoxin activation and detoxification.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-9027307', 'xref_name': 'Biosynthesis of maresin-like SPMs.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'CYP3A4', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-112043', 'xref_name': 'PLC beta mediated events.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-1489509', 'xref_name

{'CHEMBL2664': [{'xref_id': 'R-HSA-156581',
   'xref_name': 'Methylation.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1614635',
   'xref_name': 'Sulfur amino acid metabolism.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2408508',
   'xref_name': 'Metabolism of ingested SeMet, Sec, MeSec into H2Se.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5578997',
   'xref_name': 'Defective AHCY causes HMAHCHD.',
   'xref_src_db': 'Reactome'},
  {'component_synonym': 'AHCY', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL340': [{'xref_id': 'R-HSA-211981',
   'xref_name': 'Xenobiotics.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5423646',
   'xref_name': 'Aflatoxin activation and detoxification.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-9027307',
   'xref_name': 'Biosynthesis of maresin-like SPMs.',
   'xref_src_db': 'Reactome'},
  {'component_synonym': 'CYP3A4', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL3751646': [{'xref_id': 'R-HSA-112043',
   'xref_name': 'PLC b

In [24]:
pchem_act_new= chembl2gene2path(chem2uni,pchem_act)
pchem_act_new

{'CHEMBL152': [{'activity_comment': None,
   'activity_id': 302681,
   'activity_properties': [],
   'assay_chembl_id': 'CHEMBL746888',
   'assay_description': 'Inhibitory activity against replication of HCMV in NHDF cells',
   'assay_type': 'F',
   'assay_variant_accession': None,
   'assay_variant_mutation': None,
   'bao_endpoint': 'BAO_0000190',
   'bao_format': 'BAO_0000219',
   'bao_label': 'cell-based format',
   'canonical_smiles': 'Nc1ccn(C[C@@H](CO)OCP(=O)(O)O)c(=O)n1',
   'data_validity_comment': None,
   'data_validity_description': None,
   'document_chembl_id': 'CHEMBL1129529',
   'document_journal': 'J. Med. Chem.',
   'document_year': 1996,
   'ligand_efficiency': None,
   'molecule_chembl_id': 'CHEMBL152',
   'molecule_pref_name': 'CIDOFOVIR',
   'parent_molecule_chembl_id': 'CHEMBL152',
   'pchembl_value': '6.30',
   'potential_duplicate': 0,
   'qudt_units': 'http://www.openphacts.org/units/Nanomolar',
   'record_id': 37715,
   'relation': '=',
   'src_id': 1,
   'st

In [26]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [27]:
mpox_graph = chem2moa_rel_2(pchem_mech,mpox_graph)
mpox_graph = chem2dis_rel(pchem_dis,mpox_graph)
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
to_jupyter(mpox_graph)

<IPython.core.display.Javascript object>

In [28]:
mpox_graph.summarize()

---------------------  ---------------
Name                   Monkeypox Graph
Version
Number of Nodes        160
Number of Namespaces   6
Number of Edges        364
Number of Annotations  0
Number of Citations    1
Number of Authors      0
Number of Components   5
Number of Warnings     0
Network Density        1.43E-02
---------------------  ---------------

Type (6)             Count  Example
-----------------  -------  ------------------------------------------------------
Abundance               41  a(ChEMBLAssay:CHEMBL918571)
Population              36  pop(Disease:"Herpes Simplex")
miRNA                   30  m(HP:CHEMBL613738)
RNA                     30  r(Pathway:"Defective AHCY causes HMAHCHD.")
Pathology               17  path(ChEMBL:CHEMBL494759)
BiologicalProcess        6  bp(MOA:"Human herpesvirus 1 DNA polymerase inhibitor")

Prefix (6)    Name                        Count  Example
------------  ------------------------  -------  ------------------------------------------